In [1]:

using DynamicPolynomials, MultivariatePolynomials, HomotopyContinuation, JSON
include("construct_lib.jl")

# Constants and Parameters
const d1, d2, ds = 3, 5, 1  # Degree range and step
const n, a, b = 2, 1, 3
const C = a / b  # Scaling constant
const delta, alph = 1 / 2, 9 / 10  # Sampling parameters

(0.5, 0.9)

In [2]:
L2 = []
function discrete_l2_norm(P, tref, C)
    # Define the grid
    n = 100
    x_vals = range(-C, C, length=n)
    y_vals = range(-C, C, length=n)
    
    # Initialize the norm
    norm = 0.0

    # Compute the difference and sum up the squares
    for x_val in x_vals, y_val in y_vals
        P_val = subs(P, x[1] => x_val, x[2] => y_val)
        println(P_val)
        f_val = tref(x_val, y_val)
        diff = P_val - f_val
        norm += diff^2
    end

    # Take the square root and normalize by the grid size
    norm = sqrt(norm / (n * n))

    return norm
end

for (i, d) in enumerate(d1:ds:d2)
    ## Read and parse the files
    file_path_pts = expanduser("data/pts_rat_msolve_d$(d)_C_$(a)_$b.txt")
    file_path_pol = expanduser("data/pol_rat_msolve_d$(d)_C_$(a)_$b.txt")

    # Collect the polynomial
    data_pol = read(file_path_pol, String)
    # Read the polynomial approximant
    polynomial = eval(Meta.parse(data_pol))
    push!(L2, discrete_l2_norm(polynomial, tref, C))

end
println(L2)

UndefVarError: UndefVarError: `y` not defined

In [3]:
@polyvar(x, y) # Define polynomial ring
array_of_vectors_x = Vector{Vector{Float64}}()
array_of_vectors_y = Vector{Vector{Float64}}()

for (i, d) in enumerate(d1:ds:d2)
    ## Read and parse the files
    file_path_pts = expanduser("data/pts_rat_msolve_d$(d)_C_$(a)_$b.txt")
    file_path_pol = expanduser("data/pol_rat_msolve_d$(d)_C_$(a)_$b.txt")

    # Read points msolve
    data_pts = read(file_path_pts, String)
    trimmed_content = strip(data_pts, ['[', ']']) # trim brackets
    rows = split(trimmed_content, "], [") # split into two arrays
    # Collect the two arrays of msolve PlotlyKaleidopoints
    data_array = [parse.(Float64, split(strip(row, ['[', ']']), ", ")) for row in rows]

    # Collect the polynomial
    data_pol = read(file_path_pol, String)
    # Read the polynomial approximant
    polynomial = eval(Meta.parse(data_pol))

    # Compute derivatives
    P1 = differentiate(polynomial, x)
    P2 = differentiate(polynomial, y)

    # Compute Homotopy Continuation
    homoto = RR_xy_solve(n, P1, P2)

    # Define the condition for filtering
    condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1

    # Filter points using the filter function
    filtered_points = filter(condition, homoto)
    h_x = [point[1] for point in filtered_points] # for plotting
    h_y = [point[2] for point in filtered_points]
    push!(array_of_vectors_x, h_x)
    push!(array_of_vectors_y, h_y)

end


InterruptException: InterruptException:

In [4]:
function create_vector_of_pairs(vec1, vec2)
    @assert length(vec1) == length(vec2) "Vectors must be of the same length"
    return collect(zip(vec1, vec2))
end

PTS_pair = create_vector_of_pairs(array_of_vectors_x,array_of_vectors_y)
J = []
for x in PTS_pair
    push!(J, create_vector_of_pairs(x[1], x[2]))
    println(create_vector_of_pairs(x[1], x[2]))Plotl
end

Goal is to have an identification of all local minima and maxima into two arrays of points. This is to be done by evaluating f and then generating a black and white picture, where each pixel is the average change compared to evaluations of f on the direct neighbors.  

In [1]:
using Plots
plotly()  # or gr()

# Define the function and domain

# Generate the grid and evaluate the function
N = 100  # resolution of the grid
x = range(-C, C, length=N)
y = range(-C, C, length=N)
z = [tref(xi, yi) for xi in x, yi in y]

scatter!(PTS_pair, label="Homotopy Continuation Points")

# Create the plot with more contours and smooth transitions
contour(x, y, z, fill=true, color=:viridis,
    xlabel="x", ylabel="y", levels=50, c=:viridis)

┌ Warning: `PlotlyKaleido` 2.2.4 is not compatible with this version of `Plots`. The declared compatibility is 1.
└ @ Plots /home/georgy/.julia/packages/Plots/ju9dp/src/backends.jl:55


UndefVarError: UndefVarError: `C` not defined

In [ ]:
plotlyjs()

plot(x, y, z, st=:surface, title="3D plot of tref(x,y)", xlabel="x", ylabel="y", zlabel="z", color=:viridis)


In [ ]:

using Distances
plotlyjs()

# Define the function and domain

# Generate the grid and evaluate the function
N = 50  # resolution of the grid
x = range(-C, C, length=N)
y = range(-C, C, length=N)
z = [tref(xi, yi) for xi in x, yi in y]

# Convert z to a matrix for easier indexing
z_matrix = reshape(z, N, N)

# Function to find local minima and maxima
function find_extrema(z::Array{Float64,2})
    n, m = size(z)
    minima = []
    maxima = []
    for i in 2:n-1
        for j in 2:m-1
            neighborhood = z[i-1:i+1, j-1:j+1]
            center = z[i, j]
            if center == minimum(neighborhood)
                push!(minima, (i, j))
            elseif center == maximum(neighborhood)
                push!(maxima, (i, j))
            end
        end
    end
    return minima, maxima
end

# Find local minima and maxima
minima_indices, maxima_indices = find_extrema(z_matrix)

# Extract the coordinates of the minima and maxima
minima_coords = [(x[i], y[j]) for (i, j) in minima_indices]
maxima_coords = [(x[i], y[j]) for (i, j) in maxima_indices]

# Combine minima and maxima coordinates
extrema_coords = vcat(minima_coords, maxima_coords)

# Compute pairwise distances between all extrema
dist_matrix = pairwise(Euclidean(), extrema_coords)

# Find the minimum non-zero distance
min_dist = minimum(dist_matrix[dist_matrix.>0])

# The smallest radius for disjoint circles is half of the minimum distance
radius = min_dist / 2

# Create the 2D contour plot
contour(x, y, z_matrix, fill=true, color=:viridis, title="Contour plot of tref(x,y) with Minima and Maxima",
    xlabel="x", ylabel="y", levels=30, smooth=true, c=:viridis)

# Plot the minima and maxima points
scatter!(first.(minima_coords), last.(minima_coords), color="blue", marker=:circle, label="Minima")
scatter!(first.(maxima_coords), last.(maxima_coords), color="red", marker=:diamond, label="Maxima")

# Plot circles around each extremum
for coord in extrema_coords
    scatter!([coord[1]], [coord[2]], m=(0.1, :black), label=false)
    plot!([coord[1] .+ radius * cos(θ) for θ in range(0, stop=2pi, length=100)],
        [coord[2] .+ radius * sin(θ) for θ in range(0, stop=2pi, length=100)],
        color=:black, label=false)
end


Generate the discrete L2-norm of the error of $f(x,y)$ and the approximant $p(x, y)$ on a grid of size $100\times 100$